### <font color=red>1.测试环境带评估和bad case输出;(带评估.ipynb)<br>2.生产环境代码不带bad_case输出;(-copy.ipynb)<br>3.第一次预测的时间和里程,本来在标签平台没有字段的,所以可以放到备注二和备注三中!!!

In [19]:
import os,time,gc,re,math
import pyspark
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
from dateutil.relativedelta import relativedelta

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext,SparkConf,SQLContext
from pyspark.sql import Window
from pyspark.conf import SparkConf
from pyspark.ml.feature import *
from pyspark.ml import pipeline
from pyspark.ml.evaluation import *
from pyspark.sql import Row
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.linalg import Vectors,VectorUDT
from pyspark.sql.functions import mean,stddev,log,abs
from pyspark.ml.feature import MinMaxScaler,Normalizer
from pyspark.mllib.tree import GradientBoostedTrees
from pyspark.mllib.regression import LabeledPoint

from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.linalg import SparseVector
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.functions import date_format

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor

import pyspark.sql.functions as F
warnings.filterwarnings('ignore')

In [20]:
ft=time.time()

In [21]:
spark = pyspark.sql.SparkSession.builder.appName('LQ_SKD_D222_2').config('spark.shuffle.consolidateFiles','true').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

today=datetime.strftime(datetime.now(),'%Y-%m-%d')
print(today)

revise_types=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

2020-08-11


In [22]:
def check_time(s):
    print("run time is: ",(time.time()-s)/60)

def get_miss_cols(dff):
    tmp=dff.agg(*[(1-(fn.count(c)/fn.count('*'))).alias(c) for c in dff.columns])
    
    cols=tmp.columns
    leng=len(cols)
    lst=tmp.collect()

    miss_cols=[]
    for i in range(leng):
        v=[x[i] for x in lst][0]
        if v>0:
            miss_cols.append(cols[i])
    return miss_cols,tmp

def check_run_time(s):
    print("the run time is: ",(time.time()-s)/60)

def get_miss_cols(dff):
    tmp=dff.agg(*[(1-(fn.count(c)/fn.count('*'))).alias(c) for c in dff.columns])
    
    cols=tmp.columns
    leng=len(cols)
    lst=tmp.collect()

    miss_cols=[]
    for i in range(leng):
        v=[x[i] for x in lst][0]
        if v>0:
            miss_cols.append(cols[i])
    return miss_cols,tmp

def get_window_data(model_df,baoyang_id_num,baoyang_id_max):
    if baoyang_id_num==1:
        print('The baoyang_id_num should >=2 !')
        return [],[]
    
    elif (baoyang_id_num>=2) and(baoyang_id_num<baoyang_id_max):
        dt=model_df.filter(col('baoyang_times')>=baoyang_id_num)
        train1=dt.filter(col('baoyang_id')<=(baoyang_id_num-1))
        test1=model_df.filter(col('baoyang_times')==(baoyang_id_num-1))
#         print(train1.count())
#         print(test1.count())
        return train1,test1
    
    elif (baoyang_id_max>9) and (baoyang_id_num>8) and (baoyang_id_num<baoyang_id_max):
        dt=model_df.filter(col('baoyang_times')>=baoyang_id_num)
        train1=dt.filter(col('baoyang_id')<=(baoyang_id_num-1))
        test1=model_df.filter(col('baoyang_times')==(baoyang_id_num-1))
#         print(train1.count())
#         print(test1.count())
        return train1,test1

In [23]:
def get_time_feat2(data1,checkt,the_baoyang_id,end_id):
    train_cols2=[]
    
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')  #.join(checkt2,on=['vin'],how='left')

    time_feat3=['whether_discount','year', 'mon','xun','dayofweek','is_workday'] #'dayofweek', 

    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')  #.join(checkt2,on=['vin'],how='left')
    
#     time_feat4=['repair_type','repair_station'] #放CountVector中,但是速度太慢,所以舍弃
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select('vin',x).dropDuplicates().withColumnRenamed(x,'%s_1'%x)
#         train_cols2.append('%s_1'%(x))
#         checkt=checkt.join(checkt1,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
        
#     for x in ['member_type','ies_name','province','city','car_age_bin','family_name','city_class']:
#         tmp=checkt.groupBy(x).agg(sum('daydiff_1').alias('%s_daydiff_%s'%(x,'sum')),
#                                   sum('milediff_1').alias('%s_milediff_%s'%(x,'sum')),
#                                   sum('dayofmile_1').alias('%s_dayofmile_%s'%(x,'sum')),
#                                   mean('daydiff_1').alias('%s_daydiff_%s'%(x,'mean')),
#                                   mean('milediff_1').alias('%s_milediff_%s'%(x,'mean')),
#                                   mean('dayofmile_1').alias('%s_dayofmile_%s'%(x,'mean')),
#                                   min('daydiff_1').alias('%s_daydiff_%s'%(x,'min')),
#                                   min('milediff_1').alias('%s_milediff_%s'%(x,'min')),
#                                   min('dayofmile_1').alias('%s_dayofmile_%s'%(x,'min')),
#                                   max('daydiff_1').alias('%s_daydiff_%s'%(x,'max')),
#                                   max('milediff_1').alias('%s_milediff_%s'%(x,'max')),
#                                   max('dayofmile_1').alias('%s_dayofmile_%s'%(x,'max')))
#         checkt=checkt.join(tmp,on=x,how='left')
#         train_cols2=train_cols2+['%s_daydiff_%s'%(x,'sum'),'%s_milediff_%s'%(x,'sum'),'%s_dayofmile_%s'%(x,'sum'),
#                                  '%s_daydiff_%s'%(x,'mean'),'%s_milediff_%s'%(x,'mean'),'%s_dayofmile_%s'%(x,'mean'),
#                                  '%s_daydiff_%s'%(x,'min'),'%s_milediff_%s'%(x,'min'),'%s_dayofmile_%s'%(x,'min'),
#                                  '%s_daydiff_%s'%(x,'max'),'%s_milediff_%s'%(x,'max'),'%s_dayofmile_%s'%(x,'max')]
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))
        
    return checkt,train_cols2

In [24]:
def get_time_feat3(data1,checkt,the_baoyang_id,end_id):
    
    train_cols2=[]
    
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                         mean(x).alias('%s_mean'%x))
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_mean'%x)
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

    time_feat3=['whether_discount','year', 'mon','xun','dayofweek','is_workday'] #'dayofweek', 

    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x))
        checkt2=checkt2.withColumn('%s_ratio'%x,col('%s_sum'%x)/end_id) #得到历史和占比
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_ratio'%x)
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

#     time_feat4=['repair_type','repair_station']
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).agg(collect_set(x))
#         cols=checkt1.columns
#         cols.remove('vin')
#         for y in cols:
#             checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次转置
#             checkt1=checkt1.withColumn('%s_%s'%(x,y),concat_ws(',',col('%s_%s'%(x,y))))
# #             the_udf=udf(lambda x:quchong(x))
# #             checkt1=checkt1.withColumn('%s_%s'%(x,y),the_udf(col('%s_%s'%(x,y)))
#             train_cols2.append('%s_%s'%(x,y))
# #         checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(collect_set(x).alias('history_%s'%x))
# #         train_cols2.append('history_%s'%x)
#         checkt=checkt.join(checkt1,on=['vin'],how='left')#.join(checkt2,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        train_cols2.append('%s_2_1'%x)

    for x in ['daydiff','milediff','dayofmile']:
        tmp=data1.select('vin','repair_date',x).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                    mean(x).alias('%s_mean'%x),\
                                                                                    min(x).alias('%s_min'%x),\
                                                                                    max(x).alias('%s_max'%x))
        checkt=checkt.join(tmp,on=['vin'],how='left')
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_mean'%x)
        train_cols2.append('%s_min'%x)
        train_cols2.append('%s_max'%x)
        
        checkt=checkt.withColumn('%s_2_1_diff'%x,col('%s_2'%x)-col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1_inc'%x,col('%s_2_1_diff'%x)/col('%s_1'%x))
        train_cols2.append('%s_2_1_diff'%x)
        train_cols2.append('%s_2_1'%x)
        train_cols2.append('%s_2_1_inc'%x)
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_ratio_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x)).\
        withColumn('%s_ratio_2_1'%x,col('%s_1'%x)/col('repair_%s_2'%x))
        train_cols2.append('%s_ratio_2_2'%x)
        train_cols2.append('%s_ratio_2_1'%x)
    
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))

    return checkt,train_cols2

In [25]:
def get_time_feat4(data1,checkt,the_baoyang_id,end_id):
    
    train_cols2=[]
    
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                         mean(x).alias('%s_mean'%x),\
                                                                                        min(x).alias('%s_min'%x),\
                                                                                        max(x).alias('%s_max'%x))
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

    time_feat3=['whether_discount','year', 'mon','xun','dayofweek','is_workday'] #'dayofweek', 

    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x))
        checkt2=checkt2.withColumn('%s_ratio'%x,col('%s_sum'%x)/end_id) #得到历史和占比
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_ratio'%x)
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

#     time_feat4=['repair_type','repair_station']
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).agg(collect_set(x))
#         cols=checkt1.columns
#         cols.remove('vin')
#         for y in cols:
#             checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次转置
#             checkt1=checkt1.withColumn('%s_%s'%(x,y),concat_ws(',',col('%s_%s'%(x,y))))
# #             the_udf=udf(lambda x:quchong(x))
# #             checkt1=checkt1.withColumn('%s_%s'%(x,y),the_udf(col('%s_%s'%(x,y)))
#             train_cols2.append('%s_%s'%(x,y))
# #         checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(collect_set(x).alias('history_%s'%x))
# #         train_cols2.append('history_%s'%x)
#         checkt=checkt.join(checkt1,on=['vin'],how='left')#.join(checkt2,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        train_cols2.append('%s_3_2'%x)
        train_cols2.append('%s_3_1'%x)
        train_cols2.append('%s_2_1'%x)
        
    for x in ['daydiff','milediff','dayofmile']:
        tmp=data1.select('vin','repair_date',x).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                   mean(x).alias('%s_mean'%x),\
                                                                                   min(x).alias('%s_min'%x),\
                                                                                   max(x).alias('%s_max'%x))
        checkt=checkt.join(tmp,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        
        checkt=checkt.withColumn('%s_3_2_diff'%x,col('%s_3'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_diff'%x,col('%s_3'%x)-col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1_diff'%x,col('%s_2'%x)-col('%s_1'%x))
        train_cols2=train_cols2+['%s_3_2_diff'%x,'%s_3_1_diff'%x,'%s_2_1_diff'%x]
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        train_cols2=train_cols2+['%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]
        
        checkt=checkt.withColumn('%s_3_2_inc'%x,col('%s_3_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_inc'%x,col('%s_3_1_diff'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1_inc'%x,col('%s_2_1_diff'%x)/col('%s_1'%x))
        train_cols2=train_cols2+['%s_3_2_inc'%x,'%s_3_1_inc'%x,'%s_2_1_inc'%x]
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_bi_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_1'%x,col('%s_1'%x)/col('repair_%s_2'%x))
    
        checkt=checkt.withColumn('%s_bi_3_1'%x,col('%s_1'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_2'%x,col('%s_2'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_3'%x,col('%s_3'%x)/col('repair_%s_3'%x))
        
        train_cols2=train_cols2+['%s_bi_2_2'%x,'%s_bi_2_1'%x,'%s_bi_3_1'%x,
                                 '%s_bi_3_2'%x,'%s_bi_3_3'%x]
    
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))
        
    return checkt,train_cols2

In [26]:
def get_time_feat5(data1,checkt,the_baoyang_id,end_id):
    train_cols2=[]
    
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                         mean(x).alias('%s_mean'%x),\
                                                                                        min(x).alias('%s_min'%x),\
                                                                                        max(x).alias('%s_max'%x))
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

    time_feat3=['whether_discount','year', 'mon','xun','dayofweek', 'is_workday'] #'dayofweek', 

    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x))
        checkt2=checkt2.withColumn('%s_ratio'%x,col('%s_sum'%x)/end_id) #得到历史和占比
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_ratio'%x)
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

#     time_feat4=['repair_type','repair_station']
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).agg(collect_set(x))
#         cols=checkt1.columns
#         cols.remove('vin')
#         for y in cols:
#             checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次转置
#             checkt1=checkt1.withColumn('%s_%s'%(x,y),concat_ws(',',col('%s_%s'%(x,y))))
# #             the_udf=udf(lambda x:quchong(x))
# #             checkt1=checkt1.withColumn('%s_%s'%(x,y),the_udf(col('%s_%s'%(x,y)))
#             train_cols2.append('%s_%s'%(x,y))
# #         checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(collect_set(x).alias('history_%s'%x))
# #         train_cols2.append('history_%s'%x)
#         checkt=checkt.join(checkt1,on=['vin'],how='left')#.join(checkt2,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,'%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]

    for x in ['daydiff','milediff','dayofmile']:
        tmp=data1.select('vin','repair_date',x).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                   mean(x).alias('%s_mean'%x),\
                                                                                   min(x).alias('%s_min'%x),\
                                                                                   max(x).alias('%s_max'%x))
        checkt=checkt.join(tmp,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        
        checkt=checkt.withColumn('%s_4_3_diff'%x,col('%s_4'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_diff'%x,col('%s_4'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_diff'%x,col('%s_4'%x)-col('%s_1'%x))
        checkt=checkt.withColumn('%s_3_2_diff'%x,col('%s_3'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_diff'%x,col('%s_3'%x)-col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1_diff'%x,col('%s_2'%x)-col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_4_3_diff'%x,'%s_4_2_diff'%x,'%s_4_1_diff'%x,'%s_3_2_diff'%x,\
                                 '%s_3_1_diff'%x,'%s_2_1_diff'%x]
        
        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,'%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]
        
        checkt=checkt.withColumn('%s_4_3_inc'%x,col('%s_4_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_inc'%x,col('%s_4_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_inc'%x,col('%s_4_1_diff'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_3_2_inc'%x,col('%s_3_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_inc'%x,col('%s_3_1_diff'%x)/col('%s_1'%x))
        checkt=checkt.withColumn('%s_2_1_inc'%x,col('%s_2_1_diff'%x)/col('%s_1'%x))
        train_cols2=train_cols2+['%s_4_3_inc'%x,'%s_4_2_inc'%x,'%s_4_1_inc'%x,'%s_3_2_inc'%x,'%s_3_1_inc'%x,\
                                 '%s_2_1_inc'%x]
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_bi_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_1'%x,col('%s_1'%x)/col('repair_%s_2'%x))
        
        checkt=checkt.withColumn('%s_bi_3_3'%x,col('%s_3'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_2'%x,col('%s_2'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_1'%x,col('%s_1'%x)/col('repair_%s_3'%x))
        
        checkt=checkt.withColumn('%s_bi_4_4'%x,col('%s_4'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_3'%x,col('%s_3'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_2'%x,col('%s_2'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_1'%x,col('%s_1'%x)/col('repair_%s_4'%x))

        train_cols2=train_cols2+['%s_bi_2_2'%x,'%s_bi_2_1'%x,'%s_bi_3_3'%x,'%s_bi_3_2'%x,'%s_bi_3_1'%x,
                            '%s_bi_4_4'%x,'%s_bi_4_3'%x,'%s_bi_4_2'%x,'%s_bi_4_1'%x]
        
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))
        
    return checkt,train_cols2

In [27]:
def get_time_feat6(data1,checkt,the_baoyang_id,end_id):
    train_cols2=[]
    
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                         mean(x).alias('%s_mean'%x),\
                                                                                        min(x).alias('%s_min'%x),\
                                                                                        max(x).alias('%s_max'%x))
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

    time_feat3=['whether_discount','year', 'mon','xun','dayofweek', 'is_workday'] #'dayofweek', 

    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x))
        checkt2=checkt2.withColumn('%s_ratio'%x,col('%s_sum'%x)/end_id) #得到历史和占比
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_ratio'%x)
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

#     time_feat4=['repair_type','repair_station']
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).agg(collect_set(x))
#         cols=checkt1.columns
#         cols.remove('vin')
#         for y in cols:
#             checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次转置
#             checkt1=checkt1.withColumn('%s_%s'%(x,y),concat_ws(',',col('%s_%s'%(x,y))))
# #             the_udf=udf(lambda x:quchong(x))
# #             checkt1=checkt1.withColumn('%s_%s'%(x,y),the_udf(col('%s_%s'%(x,y)))
#             train_cols2.append('%s_%s'%(x,y))
# #         checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(collect_set(x).alias('history_%s'%x))
# #         train_cols2.append('history_%s'%x)
#         checkt=checkt.join(checkt1,on=['vin'],how='left')#.join(checkt2,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_5_4'%x,col('%s_5'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3'%x,col('%s_5'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2'%x,col('%s_5'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1'%x,col('%s_5'%x)/col('%s_1'%x))

        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_5_4'%x,'%s_5_3'%x,'%s_5_2'%x,'%s_5_1'%x,
                                 '%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,
                                 '%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]

    for x in ['daydiff','milediff','dayofmile']:
        tmp=data1.select('vin','repair_date',x).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                   mean(x).alias('%s_mean'%x),\
                                                                                   min(x).alias('%s_min'%x),\
                                                                                   max(x).alias('%s_max'%x))
        checkt=checkt.join(tmp,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        
        checkt=checkt.withColumn('%s_5_4_diff'%x,col('%s_5'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3_diff'%x,col('%s_5'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2_diff'%x,col('%s_5'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1_diff'%x,col('%s_5'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3_diff'%x,col('%s_4'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_diff'%x,col('%s_4'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_diff'%x,col('%s_4'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2_diff'%x,col('%s_3'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_diff'%x,col('%s_3'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1_diff'%x,col('%s_2'%x)-col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_5_4_diff'%x,'%s_5_3_diff'%x,'%s_5_2_diff'%x,'%s_5_1_diff'%x,
                                 '%s_4_3_diff'%x,'%s_4_2_diff'%x,'%s_4_1_diff'%x,
                                 '%s_3_2_diff'%x,'%s_3_1_diff'%x,'%s_2_1_diff'%x]
        
        checkt=checkt.withColumn('%s_5_4'%x,col('%s_5'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3'%x,col('%s_5'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2'%x,col('%s_5'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1'%x,col('%s_5'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_5_4'%x,'%s_5_3'%x,'%s_5_2'%x,'%s_5_1'%x,
                                 '%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,
                                 '%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]
        
        checkt=checkt.withColumn('%s_5_4_inc'%x,col('%s_5_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3_inc'%x,col('%s_5_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2_inc'%x,col('%s_5_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1_inc'%x,col('%s_5_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3_inc'%x,col('%s_4_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_inc'%x,col('%s_4_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_inc'%x,col('%s_4_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2_inc'%x,col('%s_3_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_inc'%x,col('%s_3_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1_inc'%x,col('%s_2_1_diff'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_5_4_inc'%x,'%s_5_3_inc'%x,'%s_5_2_inc'%x,'%s_5_1_inc'%x,
                                 '%s_4_3_inc'%x,'%s_4_2_inc'%x,'%s_4_1_inc'%x,
                                 '%s_3_2_inc'%x,'%s_3_1_inc'%x,'%s_2_1_inc'%x]
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_bi_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_1'%x,col('%s_1'%x)/col('repair_%s_2'%x))
        
        checkt=checkt.withColumn('%s_bi_3_3'%x,col('%s_3'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_2'%x,col('%s_2'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_1'%x,col('%s_1'%x)/col('repair_%s_3'%x))
        
        checkt=checkt.withColumn('%s_bi_4_4'%x,col('%s_4'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_3'%x,col('%s_3'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_2'%x,col('%s_2'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_1'%x,col('%s_1'%x)/col('repair_%s_4'%x))
        
        checkt=checkt.withColumn('%s_bi_5_5'%x,col('%s_5'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_4'%x,col('%s_4'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_3'%x,col('%s_3'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_2'%x,col('%s_2'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_1'%x,col('%s_1'%x)/col('repair_%s_5'%x))

        train_cols2=train_cols2+['%s_bi_5_5'%x,'%s_bi_5_4'%x,'%s_bi_5_3'%x,'%s_bi_5_2'%x,'%s_bi_5_1'%x,
                                 '%s_bi_2_2'%x,'%s_bi_2_1'%x,
                                 '%s_bi_3_3'%x,'%s_bi_3_2'%x,'%s_bi_3_1'%x,
                                 '%s_bi_4_4'%x,'%s_bi_4_3'%x,'%s_bi_4_2'%x,'%s_bi_4_1'%x]
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))
        
    return checkt,train_cols2

def get_time_feat7(data1,checkt,the_baoyang_id,end_id):
    train_cols2=[]
    
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                         mean(x).alias('%s_mean'%x),\
                                                                                        min(x).alias('%s_min'%x),\
                                                                                        max(x).alias('%s_max'%x))
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

    time_feat3=['whether_discount','year', 'mon','xun','dayofweek', 'is_workday'] #'dayofweek', 

    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x))
        checkt2=checkt2.withColumn('%s_ratio'%x,col('%s_sum'%x)/end_id) #得到历史和占比
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_ratio'%x)
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

#     time_feat4=['repair_type','repair_station']
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).agg(collect_set(x))
#         cols=checkt1.columns
#         cols.remove('vin')
#         for y in cols:
#             checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次转置
#             checkt1=checkt1.withColumn('%s_%s'%(x,y),concat_ws(',',col('%s_%s'%(x,y))))
# #             the_udf=udf(lambda x:quchong(x))
# #             checkt1=checkt1.withColumn('%s_%s'%(x,y),the_udf(col('%s_%s'%(x,y)))
#             train_cols2.append('%s_%s'%(x,y))
# #         checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(collect_set(x).alias('history_%s'%x))
# #         train_cols2.append('history_%s'%x)
#         checkt=checkt.join(checkt1,on=['vin'],how='left')#.join(checkt2,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_6_5'%x,col('%s_6'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4'%x,col('%s_6'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3'%x,col('%s_6'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2'%x,col('%s_6'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1'%x,col('%s_6'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4'%x,col('%s_5'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3'%x,col('%s_5'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2'%x,col('%s_5'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1'%x,col('%s_5'%x)/col('%s_1'%x))

        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_6_5'%x,'%s_6_4'%x,'%s_6_3'%x,'%s_6_2'%x,'%s_6_1'%x,
                                 '%s_5_4'%x,'%s_5_3'%x,'%s_5_2'%x,'%s_5_1'%x,
                                 '%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,
                                 '%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]

    for x in ['daydiff','milediff','dayofmile']:
        tmp=data1.select('vin','repair_date',x).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                   mean(x).alias('%s_mean'%x),\
                                                                                   min(x).alias('%s_min'%x),\
                                                                                   max(x).alias('%s_max'%x))
        checkt=checkt.join(tmp,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        
        checkt=checkt.withColumn('%s_6_5_diff'%x,col('%s_6'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4_diff'%x,col('%s_6'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3_diff'%x,col('%s_6'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2_diff'%x,col('%s_6'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1_diff'%x,col('%s_6'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4_diff'%x,col('%s_5'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3_diff'%x,col('%s_5'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2_diff'%x,col('%s_5'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1_diff'%x,col('%s_5'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3_diff'%x,col('%s_4'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_diff'%x,col('%s_4'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_diff'%x,col('%s_4'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2_diff'%x,col('%s_3'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_diff'%x,col('%s_3'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1_diff'%x,col('%s_2'%x)-col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_6_5_diff'%x,'%s_6_4_diff'%x,'%s_6_3_diff'%x,'%s_6_2_diff'%x,'%s_6_1_diff'%x,
                                 '%s_5_4_diff'%x,'%s_5_3_diff'%x,'%s_5_2_diff'%x,'%s_5_1_diff'%x,
                                 '%s_4_3_diff'%x,'%s_4_2_diff'%x,'%s_4_1_diff'%x,
                                 '%s_3_2_diff'%x,'%s_3_1_diff'%x,'%s_2_1_diff'%x]
        
        checkt=checkt.withColumn('%s_6_5'%x,col('%s_6'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4'%x,col('%s_6'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3'%x,col('%s_6'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2'%x,col('%s_6'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1'%x,col('%s_6'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4'%x,col('%s_5'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3'%x,col('%s_5'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2'%x,col('%s_5'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1'%x,col('%s_5'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_6_5'%x,'%s_6_4'%x,'%s_6_3'%x,'%s_6_2'%x,'%s_6_1'%x,
                                 '%s_5_4'%x,'%s_5_3'%x,'%s_5_2'%x,'%s_5_1'%x,
                                 '%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,
                                 '%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]
        
        checkt=checkt.withColumn('%s_6_5_inc'%x,col('%s_6_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4_inc'%x,col('%s_6_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3_inc'%x,col('%s_6_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2_inc'%x,col('%s_6_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1_inc'%x,col('%s_6_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4_inc'%x,col('%s_5_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3_inc'%x,col('%s_5_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2_inc'%x,col('%s_5_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1_inc'%x,col('%s_5_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3_inc'%x,col('%s_4_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_inc'%x,col('%s_4_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_inc'%x,col('%s_4_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2_inc'%x,col('%s_3_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_inc'%x,col('%s_3_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1_inc'%x,col('%s_2_1_diff'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_6_5_inc'%x,'%s_6_4_inc'%x,'%s_6_3_inc'%x,'%s_6_2_inc'%x,'%s_6_1_inc'%x,
                                 '%s_5_4_inc'%x,'%s_5_3_inc'%x,'%s_5_2_inc'%x,'%s_5_1_inc'%x,
                                 '%s_4_3_inc'%x,'%s_4_2_inc'%x,'%s_4_1_inc'%x,
                                 '%s_3_2_inc'%x,'%s_3_1_inc'%x,'%s_2_1_inc'%x]
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_bi_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_1'%x,col('%s_1'%x)/col('repair_%s_2'%x))
        
        checkt=checkt.withColumn('%s_bi_3_3'%x,col('%s_3'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_2'%x,col('%s_2'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_1'%x,col('%s_1'%x)/col('repair_%s_3'%x))
        
        checkt=checkt.withColumn('%s_bi_4_4'%x,col('%s_4'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_3'%x,col('%s_3'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_2'%x,col('%s_2'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_1'%x,col('%s_1'%x)/col('repair_%s_4'%x))
        
        checkt=checkt.withColumn('%s_bi_5_5'%x,col('%s_5'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_4'%x,col('%s_4'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_3'%x,col('%s_3'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_2'%x,col('%s_2'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_1'%x,col('%s_1'%x)/col('repair_%s_5'%x))
        
        checkt=checkt.withColumn('%s_bi_6_6'%x,col('%s_6'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_5'%x,col('%s_5'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_4'%x,col('%s_4'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_3'%x,col('%s_3'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_2'%x,col('%s_2'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_1'%x,col('%s_1'%x)/col('repair_%s_6'%x))
        
        train_cols2=train_cols2+['%s_bi_6_6'%x,'%s_bi_6_5'%x,'%s_bi_6_4'%x,'%s_bi_6_3'%x,'%s_bi_6_2'%x,'%s_bi_6_1'%x,
                                 '%s_bi_5_5'%x,'%s_bi_5_4'%x,'%s_bi_5_3'%x,'%s_bi_5_2'%x,'%s_bi_5_1'%x,
                                 '%s_bi_2_2'%x,'%s_bi_2_1'%x,
                                 '%s_bi_3_3'%x,'%s_bi_3_2'%x,'%s_bi_3_1'%x,
                                 '%s_bi_4_4'%x,'%s_bi_4_3'%x,'%s_bi_4_2'%x,'%s_bi_4_1'%x]
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))
        
    return checkt,train_cols2

def get_time_feat8(data1,checkt,the_baoyang_id,end_id):
    train_cols2=[]
    
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6,7]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                         mean(x).alias('%s_mean'%x),\
                                                                                        min(x).alias('%s_min'%x),\
                                                                                        max(x).alias('%s_max'%x))
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

    time_feat3=['whether_discount','year', 'mon','xun','dayofweek', 'is_workday'] #'dayofweek', 

    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6,7]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x))
        checkt2=checkt2.withColumn('%s_ratio'%x,col('%s_sum'%x)/end_id) #得到历史和占比
        train_cols2.append('%s_sum'%x)
        train_cols2.append('%s_ratio'%x)
        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')

#     time_feat4=['repair_type','repair_station']
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).agg(collect_set(x))
#         cols=checkt1.columns
#         cols.remove('vin')
#         for y in cols:
#             checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次转置
#             checkt1=checkt1.withColumn('%s_%s'%(x,y),concat_ws(',',col('%s_%s'%(x,y))))
# #             the_udf=udf(lambda x:quchong(x))
# #             checkt1=checkt1.withColumn('%s_%s'%(x,y),the_udf(col('%s_%s'%(x,y)))
#             train_cols2.append('%s_%s'%(x,y))
# #         checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(collect_set(x).alias('history_%s'%x))
# #         train_cols2.append('history_%s'%x)
#         checkt=checkt.join(checkt1,on=['vin'],how='left')#.join(checkt2,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6,7]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_7_6'%x,col('%s_7'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_7_5'%x,col('%s_7'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_7_4'%x,col('%s_7'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_7_3'%x,col('%s_7'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_7_2'%x,col('%s_7'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_7_1'%x,col('%s_7'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_6_5'%x,col('%s_6'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4'%x,col('%s_6'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3'%x,col('%s_6'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2'%x,col('%s_6'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1'%x,col('%s_6'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4'%x,col('%s_5'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3'%x,col('%s_5'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2'%x,col('%s_5'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1'%x,col('%s_5'%x)/col('%s_1'%x))

        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_7_6'%x,'%s_7_5'%x,'%s_7_4'%x,'%s_7_3'%x,'%s_7_2'%x,'%s_7_1'%x,
                                 '%s_6_5'%x,'%s_6_4'%x,'%s_6_3'%x,'%s_6_2'%x,'%s_6_1'%x,
                                 '%s_5_4'%x,'%s_5_3'%x,'%s_5_2'%x,'%s_5_1'%x,
                                 '%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,
                                 '%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]

    for x in ['daydiff','milediff','dayofmile']:
        tmp=data1.select('vin','repair_date',x).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),\
                                                                                   mean(x).alias('%s_mean'%x),\
                                                                                   min(x).alias('%s_min'%x),\
                                                                                   max(x).alias('%s_max'%x))
        checkt=checkt.join(tmp,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        
        checkt=checkt.withColumn('%s_7_6_diff'%x,col('%s_7'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_7_5_diff'%x,col('%s_7'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_7_4_diff'%x,col('%s_7'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_7_3_diff'%x,col('%s_7'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_7_2_diff'%x,col('%s_7'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_7_1_diff'%x,col('%s_7'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_6_5_diff'%x,col('%s_6'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4_diff'%x,col('%s_6'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3_diff'%x,col('%s_6'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2_diff'%x,col('%s_6'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1_diff'%x,col('%s_6'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4_diff'%x,col('%s_5'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3_diff'%x,col('%s_5'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2_diff'%x,col('%s_5'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1_diff'%x,col('%s_5'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3_diff'%x,col('%s_4'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_diff'%x,col('%s_4'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_diff'%x,col('%s_4'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2_diff'%x,col('%s_3'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_diff'%x,col('%s_3'%x)-col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1_diff'%x,col('%s_2'%x)-col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_7_6_diff'%x,'%s_7_5_diff'%x,'%s_7_4_diff'%x,'%s_7_3_diff'%x,'%s_7_2_diff'%x,'%s_7_1_diff'%x,
                                 '%s_6_5_diff'%x,'%s_6_4_diff'%x,'%s_6_3_diff'%x,'%s_6_2_diff'%x,'%s_6_1_diff'%x,
                                 '%s_5_4_diff'%x,'%s_5_3_diff'%x,'%s_5_2_diff'%x,'%s_5_1_diff'%x,
                                 '%s_4_3_diff'%x,'%s_4_2_diff'%x,'%s_4_1_diff'%x,
                                 '%s_3_2_diff'%x,'%s_3_1_diff'%x,'%s_2_1_diff'%x]
        
        checkt=checkt.withColumn('%s_7_6'%x,col('%s_7'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_7_5'%x,col('%s_7'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_7_4'%x,col('%s_7'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_7_3'%x,col('%s_7'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_7_2'%x,col('%s_7'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_7_1'%x,col('%s_7'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_6_5'%x,col('%s_6'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4'%x,col('%s_6'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3'%x,col('%s_6'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2'%x,col('%s_6'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1'%x,col('%s_6'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4'%x,col('%s_5'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3'%x,col('%s_5'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2'%x,col('%s_5'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1'%x,col('%s_5'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3'%x,col('%s_4'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2'%x,col('%s_4'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1'%x,col('%s_4'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2'%x,col('%s_3'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1'%x,col('%s_3'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1'%x,col('%s_2'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_7_6'%x,'%s_7_5'%x,'%s_7_4'%x,'%s_7_3'%x,'%s_7_2'%x,'%s_7_1'%x,
                                 '%s_6_5'%x,'%s_6_4'%x,'%s_6_3'%x,'%s_6_2'%x,'%s_6_1'%x,
                                 '%s_5_4'%x,'%s_5_3'%x,'%s_5_2'%x,'%s_5_1'%x,
                                 '%s_4_3'%x,'%s_4_2'%x,'%s_4_1'%x,
                                 '%s_3_2'%x,'%s_3_1'%x,'%s_2_1'%x]
        
        checkt=checkt.withColumn('%s_7_6_inc'%x,col('%s_7_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_7_5_inc'%x,col('%s_7_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_7_4_inc'%x,col('%s_7_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_7_3_inc'%x,col('%s_7_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_7_2_inc'%x,col('%s_7_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_7_1_inc'%x,col('%s_7_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_6_5_inc'%x,col('%s_6_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_6_4_inc'%x,col('%s_6_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_6_3_inc'%x,col('%s_6_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_6_2_inc'%x,col('%s_6_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_6_1_inc'%x,col('%s_6_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_5_4_inc'%x,col('%s_5_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_5_3_inc'%x,col('%s_5_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_5_2_inc'%x,col('%s_5_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_5_1_inc'%x,col('%s_5_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_4_3_inc'%x,col('%s_4_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_4_2_inc'%x,col('%s_4_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_4_1_inc'%x,col('%s_4_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_3_2_inc'%x,col('%s_3_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_3_1_inc'%x,col('%s_3_1_diff'%x)/col('%s_1'%x))
        
        checkt=checkt.withColumn('%s_2_1_inc'%x,col('%s_2_1_diff'%x)/col('%s_1'%x))
        
        train_cols2=train_cols2+['%s_7_6_inc'%x,'%s_7_5_inc'%x,'%s_7_4_inc'%x,'%s_7_3_inc'%x,'%s_7_2_inc'%x,'%s_7_1_inc'%x,
                                 '%s_6_5_inc'%x,'%s_6_4_inc'%x,'%s_6_3_inc'%x,'%s_6_2_inc'%x,'%s_6_1_inc'%x,
                                 '%s_5_4_inc'%x,'%s_5_3_inc'%x,'%s_5_2_inc'%x,'%s_5_1_inc'%x,
                                 '%s_4_3_inc'%x,'%s_4_2_inc'%x,'%s_4_1_inc'%x,
                                 '%s_3_2_inc'%x,'%s_3_1_inc'%x,'%s_2_1_inc'%x]
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_bi_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_1'%x,col('%s_1'%x)/col('repair_%s_2'%x))
        
        checkt=checkt.withColumn('%s_bi_3_3'%x,col('%s_3'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_2'%x,col('%s_2'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_1'%x,col('%s_1'%x)/col('repair_%s_3'%x))
        
        checkt=checkt.withColumn('%s_bi_4_4'%x,col('%s_4'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_3'%x,col('%s_3'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_2'%x,col('%s_2'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_1'%x,col('%s_1'%x)/col('repair_%s_4'%x))
        
        checkt=checkt.withColumn('%s_bi_5_5'%x,col('%s_5'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_4'%x,col('%s_4'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_3'%x,col('%s_3'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_2'%x,col('%s_2'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_1'%x,col('%s_1'%x)/col('repair_%s_5'%x))
        
        checkt=checkt.withColumn('%s_bi_6_6'%x,col('%s_6'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_5'%x,col('%s_5'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_4'%x,col('%s_4'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_3'%x,col('%s_3'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_2'%x,col('%s_2'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_1'%x,col('%s_1'%x)/col('repair_%s_6'%x))
        
        checkt=checkt.withColumn('%s_bi_7_1'%x,col('%s_1'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_2'%x,col('%s_2'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_3'%x,col('%s_3'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_4'%x,col('%s_4'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_5'%x,col('%s_5'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_6'%x,col('%s_6'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_7'%x,col('%s_7'%x)/col('repair_%s_7'%x))
        
        train_cols2=train_cols2+['%s_bi_7_1'%x,'%s_bi_7_2'%x,'%s_bi_7_3'%x,'%s_bi_7_4'%x,'%s_bi_7_5'%x,'%s_bi_7_6'%x,'%s_bi_7_7'%x,
                                 '%s_bi_6_6'%x,'%s_bi_6_5'%x,'%s_bi_6_4'%x,'%s_bi_6_3'%x,'%s_bi_6_2'%x,'%s_bi_6_1'%x,
                                 '%s_bi_5_5'%x,'%s_bi_5_4'%x,'%s_bi_5_3'%x,'%s_bi_5_2'%x,'%s_bi_5_1'%x,
                                 '%s_bi_2_2'%x,'%s_bi_2_1'%x,
                                 '%s_bi_3_3'%x,'%s_bi_3_2'%x,'%s_bi_3_1'%x,
                                 '%s_bi_4_4'%x,'%s_bi_4_3'%x,'%s_bi_4_2'%x,'%s_bi_4_1'%x]
    
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))

    return checkt,train_cols2

def get_time_feat_above_8(data1,checkt):
    train_cols2=[]
    
    if 'baoyang_id' in data1.columns:
        data1=data1.drop('baoyang_id')
    data1=data1.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    
    print('先求历史')
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select('vin','baoyang_id',x).groupBy('vin').agg(sum(x).alias('%s_sum'%x),mean(x).alias('%s_mean'%x),
                                                                     min(x).alias('%s_min'%x),max(x).alias('%s_max'%x))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        del checkt1
    
    print('取最近8次')
    for x in time_feat2:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6,7,8]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
        del checkt1
            
    time_feat3=['whether_discount','year', 'mon','xun', 'dayofweek', 'is_workday']
    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']  
    
    for x in time_feat3+time_feat5:
        checkt1=data1.select('vin','baoyang_id',x).dropDuplicates().groupBy('vin').agg(sum(x).alias('%s_sum'%x),mean(x).alias('%s_ratio'%x))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_ratio'%x]
        del checkt1
        
    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6,7,8]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
        del checkt1
        
#     time_feat4=['repair_type','repair_station']
#     for x in time_feat4: #每次,历史
#         checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2]).agg(collect_set(x))
#         cols=checkt1.columns
#         cols.remove('vin')
#         for y in cols:
#             checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次转置
#             checkt1=checkt1.withColumn('%s_%s'%(x,y),concat_ws(',',col('%s_%s'%(x,y))))
# #             the_udf=udf(lambda x:quchong(x))
# #             checkt1=checkt1.withColumn('%s_%s'%(x,y),the_udf(col('%s_%s'%(x,y)))
#             train_cols2.append('%s_%s'%(x,y))
# #         checkt2=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').agg(collect_set(x).alias('history_%s'%x))
# #         train_cols2.append('history_%s'%x)
#         checkt=checkt.join(checkt1,on=['vin'],how='left')#.join(checkt2,on=['vin'],how='left')

    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select('vin','baoyang_id',x).groupBy('vin').agg(sum(x).alias('%s_sum'%x),mean(x).alias('%s_mean'%x),
                                                                     min(x).alias('%s_min'%x),max(x).alias('%s_max'%x))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        del checkt1
        
    for x in time_feat1:
        checkt1=data1.select(['vin','baoyang_id',x]).dropDuplicates().groupBy('vin').pivot('baoyang_id',[1,2,3,4,5,6,7,8]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin'],how='left')
        del checkt1
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_1_2'%x,col('%s_1'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3'%x,col('%s_1'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4'%x,col('%s_1'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5'%x,col('%s_1'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6'%x,col('%s_1'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7'%x,col('%s_1'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8'%x,col('%s_1'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3'%x,col('%s_2'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4'%x,col('%s_2'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5'%x,col('%s_2'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6'%x,col('%s_2'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7'%x,col('%s_2'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8'%x,col('%s_2'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4'%x,col('%s_3'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5'%x,col('%s_3'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6'%x,col('%s_3'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7'%x,col('%s_3'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8'%x,col('%s_3'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5'%x,col('%s_4'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6'%x,col('%s_4'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7'%x,col('%s_4'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8'%x,col('%s_4'%x)/col('%s_8'%x))

        checkt=checkt.withColumn('%s_5_6'%x,col('%s_5'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7'%x,col('%s_5'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8'%x,col('%s_5'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7'%x,col('%s_6'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8'%x,col('%s_6'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8'%x,col('%s_7'%x)/col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2'%x,'%s_1_3'%x,'%s_1_4'%x,'%s_1_5'%x,'%s_1_6'%x,'%s_1_7'%x,'%s_1_8'%x,
                                 '%s_2_3'%x,'%s_2_4'%x,'%s_2_5'%x,'%s_2_6'%x,'%s_2_7'%x,'%s_2_8'%x,
                                 '%s_3_4'%x,'%s_3_5'%x,'%s_3_6'%x,'%s_3_7'%x,'%s_3_8'%x,
                                 '%s_4_5'%x,'%s_4_6'%x,'%s_4_7'%x,'%s_4_8'%x,
                                 '%s_5_6'%x,'%s_5_7'%x,'%s_5_8'%x,
                                 '%s_6_7'%x,'%s_6_8'%x,'%s_7_8'%x]

    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_1_2_diff'%x,col('%s_1'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3_diff'%x,col('%s_1'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4_diff'%x,col('%s_1'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5_diff'%x,col('%s_1'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6_diff'%x,col('%s_1'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7_diff'%x,col('%s_1'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8_diff'%x,col('%s_1'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3_diff'%x,col('%s_2'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4_diff'%x,col('%s_2'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5_diff'%x,col('%s_2'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6_diff'%x,col('%s_2'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7_diff'%x,col('%s_2'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8_diff'%x,col('%s_2'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4_diff'%x,col('%s_3'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5_diff'%x,col('%s_3'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6_diff'%x,col('%s_3'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7_diff'%x,col('%s_3'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8_diff'%x,col('%s_3'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5_diff'%x,col('%s_4'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6_diff'%x,col('%s_4'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7_diff'%x,col('%s_4'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8_diff'%x,col('%s_4'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_5_6_diff'%x,col('%s_5'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7_diff'%x,col('%s_5'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8_diff'%x,col('%s_5'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7_diff'%x,col('%s_6'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8_diff'%x,col('%s_6'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8_diff'%x,col('%s_7'%x)-col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2_diff'%x,'%s_1_3_diff'%x,'%s_1_4_diff'%x,'%s_1_5_diff'%x,'%s_1_6_diff'%x,'%s_1_7_diff'%x,'%s_1_8_diff'%x,
                                 '%s_2_3_diff'%x,'%s_2_4_diff'%x,'%s_2_5_diff'%x,'%s_2_6_diff'%x,'%s_2_7_diff'%x,'%s_2_8_diff'%x,
                                 '%s_3_4_diff'%x,'%s_3_5_diff'%x,'%s_3_6_diff'%x,'%s_3_7_diff'%x,'%s_3_8_diff'%x,
                                 '%s_4_5_diff'%x,'%s_4_6_diff'%x,'%s_4_7_diff'%x,'%s_4_8_diff'%x,
                                 '%s_5_6_diff'%x,'%s_5_7_diff'%x,'%s_5_8_diff'%x,
                                 '%s_6_7_diff'%x,'%s_6_8_diff'%x,'%s_7_8_diff'%x]
        
        checkt=checkt.withColumn('%s_1_2'%x,col('%s_1'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3'%x,col('%s_1'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4'%x,col('%s_1'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5'%x,col('%s_1'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6'%x,col('%s_1'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7'%x,col('%s_1'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8'%x,col('%s_1'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3'%x,col('%s_2'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4'%x,col('%s_2'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5'%x,col('%s_2'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6'%x,col('%s_2'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7'%x,col('%s_2'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8'%x,col('%s_2'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4'%x,col('%s_3'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5'%x,col('%s_3'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6'%x,col('%s_3'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7'%x,col('%s_3'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8'%x,col('%s_3'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5'%x,col('%s_4'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6'%x,col('%s_4'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7'%x,col('%s_4'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8'%x,col('%s_4'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_5_6'%x,col('%s_5'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7'%x,col('%s_5'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8'%x,col('%s_5'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7'%x,col('%s_6'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8'%x,col('%s_6'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8'%x,col('%s_7'%x)/col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2'%x,'%s_1_3'%x,'%s_1_4'%x,'%s_1_5'%x,'%s_1_6'%x,'%s_1_7'%x,'%s_1_8'%x,
                                 '%s_2_3'%x,'%s_2_4'%x,'%s_2_5'%x,'%s_2_6'%x,'%s_2_7'%x,'%s_2_8'%x,
                                 '%s_3_4'%x,'%s_3_5'%x,'%s_3_6'%x,'%s_3_7'%x,'%s_3_8'%x,
                                 '%s_4_5'%x,'%s_4_6'%x,'%s_4_7'%x,'%s_4_8'%x,
                                 '%s_5_6'%x,'%s_5_7'%x,'%s_5_8'%x,
                                 '%s_6_7'%x,'%s_6_8'%x,'%s_7_8'%x]
        
        checkt=checkt.withColumn('%s_1_2_inc'%x,col('%s_1_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3_inc'%x,col('%s_1_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4_inc'%x,col('%s_1_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5_inc'%x,col('%s_1_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6_inc'%x,col('%s_1_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7_inc'%x,col('%s_1_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8_inc'%x,col('%s_1_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3_inc'%x,col('%s_2_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4_inc'%x,col('%s_2_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5_inc'%x,col('%s_2_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6_inc'%x,col('%s_2_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7_inc'%x,col('%s_2_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8_inc'%x,col('%s_2_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4_inc'%x,col('%s_3_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5_inc'%x,col('%s_3_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6_inc'%x,col('%s_3_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7_inc'%x,col('%s_3_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8_inc'%x,col('%s_3_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5_inc'%x,col('%s_4_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6_inc'%x,col('%s_4_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7_inc'%x,col('%s_4_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8_inc'%x,col('%s_4_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_5_6_inc'%x,col('%s_5_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7_inc'%x,col('%s_5_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8_inc'%x,col('%s_5_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7_inc'%x,col('%s_6_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8_inc'%x,col('%s_6_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8_inc'%x,col('%s_7_8_diff'%x)/col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2_inc'%x,'%s_1_3_inc'%x,'%s_1_4_inc'%x,'%s_1_5_inc'%x,'%s_1_6_inc'%x,'%s_1_7_inc'%x,'%s_1_8_inc'%x,
                                 '%s_2_3_inc'%x,'%s_2_4_inc'%x,'%s_2_5_inc'%x,'%s_2_6_inc'%x,'%s_2_7_inc'%x,'%s_2_8_inc'%x,
                                 '%s_3_4_inc'%x,'%s_3_5_inc'%x,'%s_3_6_inc'%x,'%s_3_7_inc'%x,'%s_3_8_inc'%x,
                                 '%s_4_5_inc'%x,'%s_4_6_inc'%x,'%s_4_7_inc'%x,'%s_4_8_inc'%x,
                                 '%s_5_6_inc'%x,'%s_5_7_inc'%x,'%s_5_8_inc'%x,
                                 '%s_6_7_inc'%x,'%s_6_8_inc'%x,'%s_7_8_inc'%x]
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_bi_7_8'%x,col('%s_8'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_7'%x,col('%s_7'%x)/col('repair_%s_7'%x))
        
        checkt=checkt.withColumn('%s_bi_6_6'%x,col('%s_6'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_7'%x,col('%s_7'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_8'%x,col('%s_8'%x)/col('repair_%s_6'%x))
        
        checkt=checkt.withColumn('%s_bi_5_5'%x,col('%s_5'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_6'%x,col('%s_6'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_7'%x,col('%s_7'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_8'%x,col('%s_8'%x)/col('repair_%s_5'%x))
        
        checkt=checkt.withColumn('%s_bi_4_4'%x,col('%s_4'%x)/col('repair_%s_4'%x)) 
        checkt=checkt.withColumn('%s_bi_4_5'%x,col('%s_5'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_6'%x,col('%s_6'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_7'%x,col('%s_7'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_8'%x,col('%s_8'%x)/col('repair_%s_4'%x))  
        
        checkt=checkt.withColumn('%s_bi_3_8'%x,col('%s_8'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_7'%x,col('%s_7'%x)/col('repair_%s_3'%x))        
        checkt=checkt.withColumn('%s_bi_3_6'%x,col('%s_6'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_5'%x,col('%s_5'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_4'%x,col('%s_4'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_3'%x,col('%s_3'%x)/col('repair_%s_3'%x))
        
        checkt=checkt.withColumn('%s_bi_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_3'%x,col('%s_3'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_4'%x,col('%s_4'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_5'%x,col('%s_5'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_6'%x,col('%s_6'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_7'%x,col('%s_7'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_8'%x,col('%s_8'%x)/col('repair_%s_2'%x))
        
        checkt=checkt.withColumn('%s_bi_1_8'%x,col('%s_8'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_7'%x,col('%s_7'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_6'%x,col('%s_6'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_5'%x,col('%s_5'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_4'%x,col('%s_4'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_3'%x,col('%s_3'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_2'%x,col('%s_2'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_1'%x,col('%s_1'%x)/col('repair_%s_1'%x))
        
        train_cols2=train_cols2+['%s_bi_1_1'%x,'%s_bi_1_2'%x,'%s_bi_1_3'%x,'%s_bi_1_4'%x,'%s_bi_1_5'%x,'%s_bi_1_6'%x,'%s_bi_1_7'%x,'%s_bi_1_8'%x,
                                 '%s_bi_2_2'%x,'%s_bi_2_3'%x,'%s_bi_2_4'%x,'%s_bi_2_5'%x,'%s_bi_2_6'%x,'%s_bi_2_7'%x,'%s_bi_2_8'%x,
                                 '%s_bi_3_3'%x,'%s_bi_3_4'%x,'%s_bi_3_5'%x,'%s_bi_3_6'%x,'%s_bi_3_7'%x,'%s_bi_3_8'%x,
                                 '%s_bi_4_4'%x,'%s_bi_4_5'%x,'%s_bi_4_6'%x,'%s_bi_4_7'%x,'%s_bi_4_8'%x,
                                 '%s_bi_5_5'%x,'%s_bi_5_6'%x,'%s_bi_5_7'%x,'%s_bi_5_8'%x,
                                 '%s_bi_6_6'%x,'%s_bi_6_7'%x,'%s_bi_6_8'%x,
                                 '%s_bi_7_7'%x,'%s_bi_7_8'%x]
    
    for x in train_cols2:
        checkt=checkt.withColumn(x,col(x).cast('Double'))

    return checkt,train_cols2

In [28]:
def get_time_feat_above_8_last_batch(data1,checkt):
    train_cols2=[]
    
    if 'baoyang_id' in data1.columns:
        data1=data1.drop('baoyang_id')
    data1=data1.withColumn('baoyang_id',row_number().over(Window.partitionBy(['vin','data_mark']).orderBy(desc('repair_date'))))
    
    print('先求历史')
    time_feat2=['labor_fee','repair_amount','part_fee']
    for x in time_feat2:
        checkt1=data1.select('vin','data_mark','baoyang_id',x).groupBy(['vin','data_mark']).agg(sum(x).alias('%s_sum'%x),mean(x).alias('%s_mean'%x),
                                                                     min(x).alias('%s_min'%x),max(x).alias('%s_max'%x))
        checkt=checkt.join(checkt1,on=['vin','data_mark'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        del checkt1
    
    print('取最近8次')
    for x in time_feat2:
        checkt1=data1.select(['vin','data_mark','baoyang_id',x]).dropDuplicates().groupBy(['vin','data_mark']).pivot('baoyang_id',[1,2,3,4,5,6,7,8]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        cols.remove('data_mark')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y))
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin','data_mark'],how='left')
        del checkt1
            
    time_feat3=['whether_discount','year', 'mon','xun', 'dayofweek', 'is_workday']
    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']  
    
    for x in time_feat3+time_feat5:
        checkt1=data1.select('vin','data_mark','baoyang_id',x).dropDuplicates().groupBy(['vin','data_mark']).agg(sum(x).alias('%s_sum'%x),mean(x).alias('%s_ratio'%x))
        checkt=checkt.join(checkt1,on=['vin','data_mark'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_ratio'%x]
        del checkt1
        
    for x in time_feat3+time_feat5:
        checkt1=data1.select(['vin','data_mark','baoyang_id',x]).dropDuplicates().groupBy(['vin','data_mark']).pivot('baoyang_id',[1,2,3,4,5,6,7,8]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        cols.remove('data_mark')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin','data_mark'],how='left')
        del checkt1
        
    time_feat1=['repair_daydiff','repair_milediff','repair_dayofmile','daydiff','milediff','dayofmile']
    for x in time_feat1:
        checkt1=data1.select('vin','data_mark','baoyang_id',x).groupBy(['vin','data_mark']).agg(sum(x).alias('%s_sum'%x),mean(x).alias('%s_mean'%x),
                                                                     min(x).alias('%s_min'%x),max(x).alias('%s_max'%x))
        checkt=checkt.join(checkt1,on=['vin','data_mark'],how='left')
        train_cols2=train_cols2+['%s_sum'%x,'%s_mean'%x,'%s_min'%x,'%s_max'%x]
        del checkt1
        
    for x in time_feat1:
        checkt1=data1.select(['vin','data_mark','baoyang_id',x]).dropDuplicates().groupBy(['vin','data_mark']).pivot('baoyang_id',[1,2,3,4,5,6,7,8]).sum(x)
        cols=checkt1.columns
        cols.remove('vin')
        cols.remove('data_mark')
        for y in cols:
            checkt1=checkt1.withColumnRenamed(y,'%s_%s'%(x,y)) #每次
            train_cols2.append('%s_%s'%(x,y))
        checkt=checkt.join(checkt1,on=['vin','data_mark'],how='left')
        del checkt1
    
    for x in ['repair_daydiff','repair_milediff','repair_dayofmile']:
        checkt=checkt.withColumn('%s_1_2'%x,col('%s_1'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3'%x,col('%s_1'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4'%x,col('%s_1'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5'%x,col('%s_1'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6'%x,col('%s_1'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7'%x,col('%s_1'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8'%x,col('%s_1'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3'%x,col('%s_2'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4'%x,col('%s_2'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5'%x,col('%s_2'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6'%x,col('%s_2'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7'%x,col('%s_2'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8'%x,col('%s_2'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4'%x,col('%s_3'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5'%x,col('%s_3'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6'%x,col('%s_3'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7'%x,col('%s_3'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8'%x,col('%s_3'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5'%x,col('%s_4'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6'%x,col('%s_4'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7'%x,col('%s_4'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8'%x,col('%s_4'%x)/col('%s_8'%x))

        checkt=checkt.withColumn('%s_5_6'%x,col('%s_5'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7'%x,col('%s_5'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8'%x,col('%s_5'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7'%x,col('%s_6'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8'%x,col('%s_6'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8'%x,col('%s_7'%x)/col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2'%x,'%s_1_3'%x,'%s_1_4'%x,'%s_1_5'%x,'%s_1_6'%x,'%s_1_7'%x,'%s_1_8'%x,
                                 '%s_2_3'%x,'%s_2_4'%x,'%s_2_5'%x,'%s_2_6'%x,'%s_2_7'%x,'%s_2_8'%x,
                                 '%s_3_4'%x,'%s_3_5'%x,'%s_3_6'%x,'%s_3_7'%x,'%s_3_8'%x,
                                 '%s_4_5'%x,'%s_4_6'%x,'%s_4_7'%x,'%s_4_8'%x,
                                 '%s_5_6'%x,'%s_5_7'%x,'%s_5_8'%x,
                                 '%s_6_7'%x,'%s_6_8'%x,'%s_7_8'%x]

    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_1_2_diff'%x,col('%s_1'%x)-col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3_diff'%x,col('%s_1'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4_diff'%x,col('%s_1'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5_diff'%x,col('%s_1'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6_diff'%x,col('%s_1'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7_diff'%x,col('%s_1'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8_diff'%x,col('%s_1'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3_diff'%x,col('%s_2'%x)-col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4_diff'%x,col('%s_2'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5_diff'%x,col('%s_2'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6_diff'%x,col('%s_2'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7_diff'%x,col('%s_2'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8_diff'%x,col('%s_2'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4_diff'%x,col('%s_3'%x)-col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5_diff'%x,col('%s_3'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6_diff'%x,col('%s_3'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7_diff'%x,col('%s_3'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8_diff'%x,col('%s_3'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5_diff'%x,col('%s_4'%x)-col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6_diff'%x,col('%s_4'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7_diff'%x,col('%s_4'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8_diff'%x,col('%s_4'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_5_6_diff'%x,col('%s_5'%x)-col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7_diff'%x,col('%s_5'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8_diff'%x,col('%s_5'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7_diff'%x,col('%s_6'%x)-col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8_diff'%x,col('%s_6'%x)-col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8_diff'%x,col('%s_7'%x)-col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2_diff'%x,'%s_1_3_diff'%x,'%s_1_4_diff'%x,'%s_1_5_diff'%x,'%s_1_6_diff'%x,'%s_1_7_diff'%x,'%s_1_8_diff'%x,
                                 '%s_2_3_diff'%x,'%s_2_4_diff'%x,'%s_2_5_diff'%x,'%s_2_6_diff'%x,'%s_2_7_diff'%x,'%s_2_8_diff'%x,
                                 '%s_3_4_diff'%x,'%s_3_5_diff'%x,'%s_3_6_diff'%x,'%s_3_7_diff'%x,'%s_3_8_diff'%x,
                                 '%s_4_5_diff'%x,'%s_4_6_diff'%x,'%s_4_7_diff'%x,'%s_4_8_diff'%x,
                                 '%s_5_6_diff'%x,'%s_5_7_diff'%x,'%s_5_8_diff'%x,
                                 '%s_6_7_diff'%x,'%s_6_8_diff'%x,'%s_7_8_diff'%x]
        
        checkt=checkt.withColumn('%s_1_2'%x,col('%s_1'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3'%x,col('%s_1'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4'%x,col('%s_1'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5'%x,col('%s_1'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6'%x,col('%s_1'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7'%x,col('%s_1'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8'%x,col('%s_1'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3'%x,col('%s_2'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4'%x,col('%s_2'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5'%x,col('%s_2'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6'%x,col('%s_2'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7'%x,col('%s_2'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8'%x,col('%s_2'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4'%x,col('%s_3'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5'%x,col('%s_3'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6'%x,col('%s_3'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7'%x,col('%s_3'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8'%x,col('%s_3'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5'%x,col('%s_4'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6'%x,col('%s_4'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7'%x,col('%s_4'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8'%x,col('%s_4'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_5_6'%x,col('%s_5'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7'%x,col('%s_5'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8'%x,col('%s_5'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7'%x,col('%s_6'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8'%x,col('%s_6'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8'%x,col('%s_7'%x)/col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2'%x,'%s_1_3'%x,'%s_1_4'%x,'%s_1_5'%x,'%s_1_6'%x,'%s_1_7'%x,'%s_1_8'%x,
                                 '%s_2_3'%x,'%s_2_4'%x,'%s_2_5'%x,'%s_2_6'%x,'%s_2_7'%x,'%s_2_8'%x,
                                 '%s_3_4'%x,'%s_3_5'%x,'%s_3_6'%x,'%s_3_7'%x,'%s_3_8'%x,
                                 '%s_4_5'%x,'%s_4_6'%x,'%s_4_7'%x,'%s_4_8'%x,
                                 '%s_5_6'%x,'%s_5_7'%x,'%s_5_8'%x,
                                 '%s_6_7'%x,'%s_6_8'%x,'%s_7_8'%x]
        
        checkt=checkt.withColumn('%s_1_2_inc'%x,col('%s_1_2_diff'%x)/col('%s_2'%x))
        checkt=checkt.withColumn('%s_1_3_inc'%x,col('%s_1_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_1_4_inc'%x,col('%s_1_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_1_5_inc'%x,col('%s_1_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_1_6_inc'%x,col('%s_1_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_1_7_inc'%x,col('%s_1_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_1_8_inc'%x,col('%s_1_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_2_3_inc'%x,col('%s_2_3_diff'%x)/col('%s_3'%x))
        checkt=checkt.withColumn('%s_2_4_inc'%x,col('%s_2_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_2_5_inc'%x,col('%s_2_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_2_6_inc'%x,col('%s_2_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_2_7_inc'%x,col('%s_2_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_2_8_inc'%x,col('%s_2_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_3_4_inc'%x,col('%s_3_4_diff'%x)/col('%s_4'%x))
        checkt=checkt.withColumn('%s_3_5_inc'%x,col('%s_3_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_3_6_inc'%x,col('%s_3_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_3_7_inc'%x,col('%s_3_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_3_8_inc'%x,col('%s_3_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_4_5_inc'%x,col('%s_4_5_diff'%x)/col('%s_5'%x))
        checkt=checkt.withColumn('%s_4_6_inc'%x,col('%s_4_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_4_7_inc'%x,col('%s_4_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_4_8_inc'%x,col('%s_4_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_5_6_inc'%x,col('%s_5_6_diff'%x)/col('%s_6'%x))
        checkt=checkt.withColumn('%s_5_7_inc'%x,col('%s_5_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_5_8_inc'%x,col('%s_5_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_6_7_inc'%x,col('%s_6_7_diff'%x)/col('%s_7'%x))
        checkt=checkt.withColumn('%s_6_8_inc'%x,col('%s_6_8_diff'%x)/col('%s_8'%x))
        
        checkt=checkt.withColumn('%s_7_8_inc'%x,col('%s_7_8_diff'%x)/col('%s_8'%x))
        
        train_cols2=train_cols2+['%s_1_2_inc'%x,'%s_1_3_inc'%x,'%s_1_4_inc'%x,'%s_1_5_inc'%x,'%s_1_6_inc'%x,'%s_1_7_inc'%x,'%s_1_8_inc'%x,
                                 '%s_2_3_inc'%x,'%s_2_4_inc'%x,'%s_2_5_inc'%x,'%s_2_6_inc'%x,'%s_2_7_inc'%x,'%s_2_8_inc'%x,
                                 '%s_3_4_inc'%x,'%s_3_5_inc'%x,'%s_3_6_inc'%x,'%s_3_7_inc'%x,'%s_3_8_inc'%x,
                                 '%s_4_5_inc'%x,'%s_4_6_inc'%x,'%s_4_7_inc'%x,'%s_4_8_inc'%x,
                                 '%s_5_6_inc'%x,'%s_5_7_inc'%x,'%s_5_8_inc'%x,
                                 '%s_6_7_inc'%x,'%s_6_8_inc'%x,'%s_7_8_inc'%x]
    
    for x in ['daydiff','milediff','dayofmile']:
        checkt=checkt.withColumn('%s_bi_7_8'%x,col('%s_8'%x)/col('repair_%s_7'%x))
        checkt=checkt.withColumn('%s_bi_7_7'%x,col('%s_7'%x)/col('repair_%s_7'%x))
        
        checkt=checkt.withColumn('%s_bi_6_6'%x,col('%s_6'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_7'%x,col('%s_7'%x)/col('repair_%s_6'%x))
        checkt=checkt.withColumn('%s_bi_6_8'%x,col('%s_8'%x)/col('repair_%s_6'%x))
        
        checkt=checkt.withColumn('%s_bi_5_5'%x,col('%s_5'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_6'%x,col('%s_6'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_7'%x,col('%s_7'%x)/col('repair_%s_5'%x))
        checkt=checkt.withColumn('%s_bi_5_8'%x,col('%s_8'%x)/col('repair_%s_5'%x))
        
        checkt=checkt.withColumn('%s_bi_4_4'%x,col('%s_4'%x)/col('repair_%s_4'%x)) 
        checkt=checkt.withColumn('%s_bi_4_5'%x,col('%s_5'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_6'%x,col('%s_6'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_7'%x,col('%s_7'%x)/col('repair_%s_4'%x))
        checkt=checkt.withColumn('%s_bi_4_8'%x,col('%s_8'%x)/col('repair_%s_4'%x))  
        
        checkt=checkt.withColumn('%s_bi_3_8'%x,col('%s_8'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_7'%x,col('%s_7'%x)/col('repair_%s_3'%x))        
        checkt=checkt.withColumn('%s_bi_3_6'%x,col('%s_6'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_5'%x,col('%s_5'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_4'%x,col('%s_4'%x)/col('repair_%s_3'%x))
        checkt=checkt.withColumn('%s_bi_3_3'%x,col('%s_3'%x)/col('repair_%s_3'%x))
        
        checkt=checkt.withColumn('%s_bi_2_2'%x,col('%s_2'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_3'%x,col('%s_3'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_4'%x,col('%s_4'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_5'%x,col('%s_5'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_6'%x,col('%s_6'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_7'%x,col('%s_7'%x)/col('repair_%s_2'%x))
        checkt=checkt.withColumn('%s_bi_2_8'%x,col('%s_8'%x)/col('repair_%s_2'%x))
        
        checkt=checkt.withColumn('%s_bi_1_8'%x,col('%s_8'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_7'%x,col('%s_7'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_6'%x,col('%s_6'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_5'%x,col('%s_5'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_4'%x,col('%s_4'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_3'%x,col('%s_3'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_2'%x,col('%s_2'%x)/col('repair_%s_1'%x))
        checkt=checkt.withColumn('%s_bi_1_1'%x,col('%s_1'%x)/col('repair_%s_1'%x))
        
        train_cols2=train_cols2+['%s_bi_1_1'%x,'%s_bi_1_2'%x,'%s_bi_1_3'%x,'%s_bi_1_4'%x,'%s_bi_1_5'%x,'%s_bi_1_6'%x,'%s_bi_1_7'%x,'%s_bi_1_8'%x,
                                 '%s_bi_2_2'%x,'%s_bi_2_3'%x,'%s_bi_2_4'%x,'%s_bi_2_5'%x,'%s_bi_2_6'%x,'%s_bi_2_7'%x,'%s_bi_2_8'%x,
                                 '%s_bi_3_3'%x,'%s_bi_3_4'%x,'%s_bi_3_5'%x,'%s_bi_3_6'%x,'%s_bi_3_7'%x,'%s_bi_3_8'%x,
                                 '%s_bi_4_4'%x,'%s_bi_4_5'%x,'%s_bi_4_6'%x,'%s_bi_4_7'%x,'%s_bi_4_8'%x,
                                 '%s_bi_5_5'%x,'%s_bi_5_6'%x,'%s_bi_5_7'%x,'%s_bi_5_8'%x,
                                 '%s_bi_6_6'%x,'%s_bi_6_7'%x,'%s_bi_6_8'%x,
                                 '%s_bi_7_7'%x,'%s_bi_7_8'%x]
    
#     for x in train_cols2:
#         checkt=checkt.withColumn(x,col(x).cast('Double'))

    return checkt,train_cols2

In [29]:
def model_run(trainData1,valiData1,traindff,testdff,label_name):
#     label_name='daydiff'
    
    clf_day2=GBTRegressor(featuresCol='features', labelCol='%s_next'%label_name, 
                          predictionCol='%s_prediction_abs'%label_name, maxBins=1000,lossType='absolute')
    model_day2=clf_day2.fit(trainData1)
    
    trainData1=model_day2.transform(trainData1)
    valiData1=model_day2.transform(valiData1)
    traindff=model_day2.transform(traindff)
    testdff=model_day2.transform(testdff) #得到结果

    evaluator_mae_2 = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="mae", predictionCol='%s_prediction_abs'%label_name)
    evaluator_mse_2 = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="mse", predictionCol='%s_prediction_abs'%label_name)
    evaluator_r2_2 = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="r2", predictionCol='%s_prediction_abs'%label_name)

#     train_mae_2 = evaluator_mae_2.evaluate(trainData1)
#     print(train_mae_2)
#     train_mse_2 = evaluator_mse_2.evaluate(trainData1)
#     print(train_mse_2)
#     train_r_2 = evaluator_r2_2.evaluate(trainData1)
#     print(train_r_2)
#     print('-'*10)

#     vali_mae_2 = evaluator_mae_2.evaluate(valiData1)
#     print(vali_mae_2)
#     vali_mse_2 = evaluator_mse_2.evaluate(valiData1)
#     print(vali_mse_2)
#     vali_r_2 = evaluator_r2_2.evaluate(valiData1)
#     print(vali_r_2)
        
#     print('-'*30)
    clf_day4=RandomForestRegressor(featuresCol='features',labelCol='%s_next'%label_name, 
                                       predictionCol='%s_prediction_rf'%label_name,maxBins=1000)

    model_day4=clf_day4.fit(trainData1)
    
    trainData1=model_day4.transform(trainData1)
    valiData1=model_day4.transform(valiData1)
               
    traindff=model_day4.transform(traindff)
    testdff=model_day4.transform(testdff) #得到结果

    evaluator_mae_4 = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="mae", predictionCol='%s_prediction_rf'%label_name)
    evaluator_mse_4 = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="mse", predictionCol='%s_prediction_rf'%label_name)
    evaluator_r2_4 = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="r2", predictionCol='%s_prediction_rf'%label_name)

#     train_mae_4 = evaluator_mae_4.evaluate(trainData1)
#     print(train_mae_4)
#     train_mse_4 = evaluator_mse_4.evaluate(trainData1)
#     print(train_mse_4)
#     train_r_4= evaluator_r2_4.evaluate(trainData1)
#     print(train_r_4)
#     print('-'*10)

#     vali_mae_4 = evaluator_mae_4.evaluate(valiData1)
#     print(vali_mae_4)
#     vali_mse_4 = evaluator_mse_4.evaluate(valiData1)
#     print(vali_mse_4)
#     vali_r_4= evaluator_r2_4.evaluate(valiData1)
#     print(vali_r_4)
    
#     print('-'*30)
    trainData1=trainData1.withColumn('%s_prediction_ens'%label_name,col('%s_prediction_abs'%label_name)+col('%s_prediction_rf'%label_name)).\
    withColumn('%s_prediction_ens'%label_name,col('%s_prediction_ens'%label_name)/2)
    
    valiData1=valiData1.withColumn('%s_prediction_ens'%label_name,col('%s_prediction_abs'%label_name)+col('%s_prediction_rf'%label_name)).\
    withColumn('%s_prediction_ens'%label_name,col('%s_prediction_ens'%label_name)/2)
               
    evaluator_mae_e = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="mae", predictionCol='%s_prediction_ens'%label_name)
    evaluator_mse_e = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="mse", predictionCol='%s_prediction_ens'%label_name)
    evaluator_r2_e = RegressionEvaluator(labelCol='%s_next'%label_name, metricName="r2", predictionCol='%s_prediction_ens'%label_name)

    train_mae_e = evaluator_mae_e.evaluate(trainData1)
    print(train_mae_e)
    train_mse_e = evaluator_mse_e.evaluate(trainData1)
    print(train_mse_e)
    train_r_e = evaluator_r2_e.evaluate(trainData1)
    print(train_r_e)
    print('-'*10)

    vali_mae_e = evaluator_mae_e.evaluate(valiData1)
    print(vali_mae_e)
    vali_mse_e = evaluator_mse_e.evaluate(valiData1)
    print(vali_mse_e)
    vali_r_e = evaluator_r2_e.evaluate(valiData1)
    print(vali_r_e)
    
    traindff=traindff.withColumn('%s_prediction_ens'%label_name,col('%s_prediction_abs'%label_name)+col('%s_prediction_rf'%label_name)).\
    withColumn('%s_prediction_ens'%label_name,col('%s_prediction_ens'%label_name)/2)
    testdff=testdff.withColumn('%s_prediction_ens'%label_name,col('%s_prediction_abs'%label_name)+col('%s_prediction_rf'%label_name)).\
    withColumn('%s_prediction_ens'%label_name,col('%s_prediction_ens'%label_name)/2)

    return traindff,testdff

In [30]:
def get_fin_pred_result_mile_dayofmile(dff,is_train):
    dff=dff.withColumn('daydiff_prediction_ens',col('milediff_prediction_ens')/col('dayofmile_prediction_ens')).\
    withColumn('daydiff_prediction_ens',ceil('daydiff_prediction_ens')).\
    withColumn('daydiff_diff',col('daydiff_prediction_ens')-col('last_repair_daydiff')).\
    withColumn('milediff_diff',col('milediff_prediction_ens')-col('last_repair_milediff')).\
    withColumn('fin_mileofday',col('milediff_diff')/col('daydiff_diff'))

    def get_dates(x,y):
        a = x + relativedelta(days = int(y))
        return a
    get_dates_udf = udf(lambda x,y:get_dates(x,y),DateType())
    
    dff=dff.withColumn('purchase_date',to_date(col('purchase_date'))).\
    withColumn('next_repair_date',get_dates_udf(col('purchase_date'),col('daydiff_prediction_ens'))).\
    withColumn('next_repair_mile',ceil('milediff_prediction_ens')).\
    withColumn('last_repair_date',get_dates_udf(col('purchase_date'),col('last_repair_daydiff'))).\
    withColumn('last_repair_mile',ceil('last_repair_milediff'))
    
    if is_train=='True':
        dff=dff.withColumn('actual_repair_date',get_dates_udf(col('purchase_date'),col('daydiff_next'))).\
        withColumn('actual_repair_mile',ceil('milediff_next'))
        return dff
    else:
        return dff

    
def get_abs_bias(df):
    df=df.withColumn('day_bias',col('daydiff_prediction_ens')-col('daydiff_next')).\
    withColumn('mile_bias',col('milediff_prediction_ens')-col('milediff_next')).\
    withColumn('day_bias_abs',abs(col('day_bias'))).\
    withColumn('mile_bias_abs',abs(col('mile_bias')))

    df=df.withColumn('daydiff_group',when(col('day_bias_abs')<=10,lit('(1):0-10天')).\
                     when((col('day_bias_abs')>10)&(col('day_bias_abs')<=20),lit('(2):10-20天')).\
                     when((col('day_bias_abs')>20)&(col('day_bias_abs')<=40),lit('(3):20-40天')).\
                     when((col('day_bias_abs')>40)&(col('day_bias_abs')<=60),lit('(4):40-60天')).\
                     when((col('day_bias_abs')>60)&(col('day_bias_abs')<=90),lit('(5):60-90天')).otherwise(lit('(6):90天以上')))

    df=df.withColumn('milediff_group',when(col('mile_bias_abs')<=500,lit('(1):0-500公里')).\
                     when((col('mile_bias_abs')>500)&(col('mile_bias_abs')<=1000),lit('(2):500-1000公里')).\
                     when((col('mile_bias_abs')>1000)&(col('mile_bias_abs')<=2000),lit('(3):1000-2000公里')).\
                     when((col('mile_bias_abs')>2000)&(col('mile_bias_abs')<=4000),lit('(4):2000-4000公里')).\
                     when((col('mile_bias_abs')>4000)&(col('mile_bias_abs')<=8000),lit('(5):4000-8000公里')).otherwise(lit('(6):8000公里以上')))

    print('#################天数偏差################')
    tmp=df.select('vin','daydiff_group').dropDuplicates()
    all_vin=tmp.select('vin').distinct().count()
    tmpp=tmp.groupBy('daydiff_group').agg(count('vin').alias('人数'))
    tmpp=tmpp.withColumn('人数占比',col('人数')/all_vin)
    tmpp=tmpp.sort(asc('daydiff_group')).coalesce(1)
    print(tmpp.show(20))
    
    print('#################公里数偏差################')
    tmp=df.select('vin','milediff_group').dropDuplicates()
    tmpp=tmp.groupBy('milediff_group').agg(count('vin').alias('人数'))
    tmpp=tmpp.withColumn('人数占比',col('人数')/all_vin)
    tmpp=tmpp.sort(asc('milediff_group')).coalesce(1)
    print(tmpp.show(20)) 

In [31]:
def get_pred(baoyangdt,the_baoyang_id,baoyang_id_max):
    s=time.time()
    
    end_id=the_baoyang_id-1
    
    train1,test1=get_window_data(baoyangdt,the_baoyang_id,baoyang_id_max)
    
    train1_label=train1.select('vin','next_repair_date','next_mile').groupBy('vin').agg(max('next_repair_date').alias('fin_next_repair_date'),max('next_mile').alias('fin_next_mile'))

    train1=train1.join(train1_label,on=['vin'],how='left').withColumn('daydiff_next',datediff(col('fin_next_repair_date'),col('purchase_date'))).\
    withColumn('milediff_next',col('fin_next_mile')).drop('fin_next_repair_date','fin_next_mile','next_repair_date','next_mile')
    train1=train1.withColumn('dayofmile_next',col('milediff_next')/col('daydiff_next'))
    train1=train1.dropDuplicates()
    train1=train1.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    train1=broadcast(train1)
    print(train1.count())
    
    test1=test1.drop('next_repair_date','next_mile').\
    withColumn('milediff_next',lit(-1)).withColumn('daydiff_next',lit(-1)).withColumn('dayofmile_next',lit(-1))
    test1=test1.dropDuplicates()
    test1=test1.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    test1=broadcast(test1)
    print(test1.count())

#     print('###########合并train and test#############')
    same=[x for x in train1.columns if x in test1.columns]
    data1=train1.select(same).unionAll(test1.select(same))
    data1=data1.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    data1=broadcast(data1)
    
    cat_feat1=['member_type','member_status','ies_name','whether_bind_wechat','gender_code','city_class'] #label +  onehot
    cat_feat2=['province','city','user_tag','belong_dealer_code'] #label
    cat_feat3=['buy_id','car_age_bin'] #直接用
    cat_feat4=['family_name','buy_year','member_age'] #onehot
    cat_feat5=['car_body_type','gearbox','crew_protection_system','car_class','output_year','assembly_factory'] #label+onehot   

    time_feat1=['repair_daydiff', 'repair_milediff', 'repair_dayofmile', 'daydiff', 'milediff', 'dayofmile']
    time_feat2=['repair_amount', 'labor_fee', 'part_fee'] 
    time_feat3=['whether_discount','year', 'mon','xun', 'dayofweek', 'is_workday']
    time_feat4=['repair_type']
    time_feat5=['claim','routine_maintenance','accessory','accident_repair','general_repair',
                'first_maintance','service_act']  

    label_feat=['daydiff_next', 'milediff_next','dayofmile_next']
    
    if the_baoyang_id==2:
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat2(data1,checkt,the_baoyang_id,end_id)

    elif the_baoyang_id==3:
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat3(data1,checkt,the_baoyang_id,end_id)
  
    elif the_baoyang_id==4:
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat4(data1,checkt,the_baoyang_id,end_id)
    
    elif the_baoyang_id==5:
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat5(data1,checkt,the_baoyang_id,end_id)

    elif the_baoyang_id==6:
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat6(data1,checkt,the_baoyang_id,end_id)
        
    elif the_baoyang_id==7:
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat7(data1,checkt,the_baoyang_id,end_id)
        
    elif the_baoyang_id==8:
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat8(data1,checkt,the_baoyang_id,end_id)
        
    elif (baoyang_id_max>9) and (the_baoyang_id>=9) and (the_baoyang_id<baoyang_id_max):
        checkt=data1.select(['vin']+cat_feat1+cat_feat2+cat_feat3+cat_feat4+cat_feat5+label_feat).dropDuplicates()
        checkt,train_cols2=get_time_feat_above_8(data1,checkt)
                
    checkt=checkt.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    checkt=broadcast(checkt)   

    del train1
    del test1
    del train1_label

#     print('############增加4个重要的特征值,不放入训练,用来check bad case##########')
    if (the_baoyang_id>=2) and (the_baoyang_id<=8):
        checkt1=data1.select(['vin','purchase_date']).dropDuplicates()
        checkt2=checkt.select(['vin','repair_daydiff_%d'%(end_id),'repair_milediff_%d'%(end_id)]).dropDuplicates().\
        withColumnRenamed('repair_daydiff_%d'%(end_id),'last_repair_daydiff').\
        withColumnRenamed('repair_milediff_%d'%(end_id),'last_repair_milediff')

        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')
        
    elif (baoyang_id_max>9) and (the_baoyang_id>8) and (the_baoyang_id<baoyang_id_max):
        checkt1=data1.select(['vin','purchase_date']).dropDuplicates()
        checkt2=checkt.select(['vin','repair_daydiff_1','repair_milediff_1']).dropDuplicates().\
        withColumnRenamed('repair_daydiff_1','last_repair_daydiff').\
        withColumnRenamed('repair_milediff_1','last_repair_milediff')

        checkt=checkt.join(checkt1,on=['vin'],how='left').join(checkt2,on=['vin'],how='left')
            
    checkt=checkt.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    checkt=broadcast(checkt)
        
    del checkt1
    del checkt2
    del data1
    
    print('############开始LabelEncoder以及特征工程##########')
    checkt=checkt.withColumn('buy_id',col('buy_id').cast('Int')).\
    withColumn('car_age_bin',col('car_age_bin').cast('Int')).\
    withColumn('family_name',col('family_name').cast('Int')).\
    withColumn('city_class',col('city_class').cast('Int')).\
    withColumn('buy_year',col('buy_year').cast('Int')).\
    withColumn('member_age',col('member_age').cast('Int'))
    
    stringer = [StringIndexer(inputCol=x,outputCol='%s_index'%x) for x in cat_feat1+cat_feat2+cat_feat5] #转成labelencoder

    train_cols1= ['%s_index'%x for x in cat_feat1+cat_feat2+cat_feat5]+cat_feat3+cat_feat4

    checkt=checkt.na.fill(0)
    
    train_cols=train_cols1+train_cols2
    print(len(train_cols))

    pipeline=Pipeline(stages=stringer)
    pipeline_day1=pipeline.fit(checkt)
    checkt=pipeline_day1.transform(checkt)

    vecAssembler=VectorAssembler(inputCols=train_cols,outputCol='features')
    pipeline1=Pipeline(stages=[vecAssembler])
    pipeline_day2=pipeline1.fit(checkt)
    checkt=pipeline_day2.transform(checkt)
        
    print('###########提取训练集和测试集##############')
    traindf=checkt.filter(col('daydiff_next')!=-1)
    print(traindf.count())
    testdf=checkt.filter(col('daydiff_next')==-1)
    print(testdf.count())

    y1='daydiff_next'
    y2='milediff_next'
    y3='dayofmile_next'
    
    train_choosed=['vin','features','last_repair_daydiff','last_repair_milediff','purchase_date',y1,y2,y3]
    traindff=traindf.select(train_choosed)

    test_choosed=['vin','features','last_repair_daydiff','last_repair_milediff','purchase_date']
    testdff=testdf.select(test_choosed)
    
    traindff=traindff.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    traindff=broadcast(traindff)
    testdff=testdff.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    testdff=broadcast(testdff)

    trainData1,valiData1=traindff.randomSplit([0.7,0.3], 2020)
    
#     print('###############################天数模型####################################')
#     label_name='daydiff'
#     traindff,testdff=model_run(trainData1,valiData1,traindff,testdff,label_name)
    
    print('###############################公里数模型####################################')
    label_name='milediff'
    traindff,testdff=model_run(trainData1,valiData1,traindff,testdff,label_name)
#     testdff=model_run(trainData1,valiData1,traindff,testdff,label_name)
    
    print('###############################日均模型####################################')
    label_name='dayofmile'
    traindff,testdff=model_run(trainData1,valiData1,traindff,testdff,label_name)
#     testdff=model_run(trainData1,valiData1,traindff,testdff,label_name)
    
    train_cols=['vin','features','last_repair_daydiff','last_repair_milediff','purchase_date',y1,y2,y3]+\
    ['dayofmile_prediction_ens','milediff_prediction_ens']
    traindff_fin=traindff.select(train_cols)

    test_cols=['vin','features','last_repair_daydiff','last_repair_milediff','purchase_date']+\
    ['dayofmile_prediction_ens','milediff_prediction_ens']
    testdff_fin=testdff.select(test_cols)
    
    del traindff
    del testdff
    del trainData1
    del valiData1
    
    traindff_fin=get_fin_pred_result_mile_dayofmile(traindff_fin,'True')
    get_abs_bias(traindff_fin)
    
    testdff_fin=get_fin_pred_result_mile_dayofmile(testdff_fin,'False')
       
    print('all run time is:',(time.time()-s)/60)
    
    return testdff_fin

In [32]:
ft=time.time()

In [33]:
baoyang_model=spark.sql('select * from clms.skd_baoyang_model')
# print(baoyang_model.count())
# print(baoyang_model.select('vin').distinct().count())
# print(baoyang_model.show(1))

In [34]:
print('查看人群数量')
tmp=baoyang_model.select('vin','baoyang_times').dropDuplicates().groupBy('baoyang_times').agg(count('vin').alias('cnt'))
tmpp=tmp.filter(col('cnt')>=1000).withColumn('group',lit('-')).groupBy('group').agg(max('baoyang_times'))
baoyang_id_max=[x[0] for x in tmpp.select('max(baoyang_times)').distinct().collect()][0]
baoyang_id_max=baoyang_id_max+1
print('原先的baoyang_id_max是: ',baoyang_id_max)

查看人群数量
原先的baoyang_id_max是:  11.0


In [ ]:
if baoyang_id_max<=8:
    baoyang_id_max=9
baoyang_id_max=int(baoyang_id_max)
print(baoyang_id_max)

11


In [ ]:
the_baoyang_id=8
fin=get_pred(baoyang_model,the_baoyang_id,baoyang_id_max)
fin=fin.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin=broadcast(fin)

56056
24668
############开始LabelEncoder以及特征工程##########
558
###########提取训练集和测试集##############
8008
3524
###############################公里数模型####################################
2491.789143134471
18183497.195829958
0.9465176643283197
----------
3843.9058744461504
55617123.89676841
0.8529987354146511
###############################日均模型####################################


In [ ]:
for the_baoyang_id in range(9,baoyang_id_max):
    print('----------------------------------%d------------------------------------'%the_baoyang_id)
    fin_tmp=get_pred(baoyang_model,the_baoyang_id,baoyang_id_max)
    fin=fin.unionAll(fin_tmp)
    fin_tmp.unpersist()
    fin=fin.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    fin=broadcast(fin)

In [ ]:
# test_cols=['vin', 'last_repair_daydiff', 'last_repair_milediff', 'purchase_date', 'daydiff_prediction_ens',
#            'dayofmile_prediction_ens', 'milediff_prediction_ens', 'milediff_diff', 'daydiff_diff',
#            'fin_mileofday', 'next_repair_date', 'next_repair_mile', 'last_repair_date', 'last_repair_mile']
# fin_1=fin.select(test_cols)

# fin_1.toPandas().to_excel('model_result_8_10.xlsx',index=False,encoding='gbk')

# fin_1.registerTempTable('next_model_result_reg')
# spark.sql('drop table if exists clms.next_model_result_2')
# spark.sql('create table if not exists clms.next_model_result_2 (vin string,last_repair_daydiff double,\
# last_repair_milediff double,purchase_date date,daydiff_prediction_ens double,dayofmile_prediction_ens double,\
# milediff_prediction_ens double,milediff_diff double,daydiff_diff double,fin_mileofday double,\
# next_repair_date date,next_repair_mile double,last_repair_date date,last_repair_mile double)')
# spark.sql('insert overwrite table clms.next_model_result_2 select * from next_model_result_reg')

# print((time.time()-ft)/60)

In [ ]:
fin.toPandas().to_excel('model_result_8_10.xlsx',index=False,encoding='gbk')

In [ ]:
print('################################')
test_cols=['vin', 'last_repair_daydiff', 'last_repair_milediff', 'purchase_date', 'daydiff_prediction_ens',
           'dayofmile_prediction_ens', 'milediff_prediction_ens', 'milediff_diff', 'daydiff_diff',
           'fin_mileofday', 'next_repair_date', 'next_repair_mile', 'last_repair_date', 'last_repair_mile']
fin=fin.select(test_cols)

fin.registerTempTable('next_model_result_reg')
spark.sql('drop table if exists clms.next_model_result_2')
spark.sql('create table if not exists clms.next_model_result_2 (vin string,last_repair_daydiff double,\
last_repair_milediff double,purchase_date date,daydiff_prediction_ens double,dayofmile_prediction_ens double,\
milediff_prediction_ens double,milediff_diff double,daydiff_diff double,fin_mileofday double,\
next_repair_date date,next_repair_mile double,last_repair_date date,last_repair_mile double)')
spark.sql('insert overwrite table clms.next_model_result_2 select * from next_model_result_reg')

In [ ]:
print((time.time()-ft)/60)